In [8]:
# -- IMPORTS -- #
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.layers import Input, InputLayer, Dense, Conv2D, MaxPooling2D, Flatten, Activation
from keras.models import Sequential, Model
from keras.preprocessing import image as kerasImage
import keras.activations as kerasAct
import PIL.Image as pilImage
import keras.backend as K
import numpy as np
import os
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
# -- DEFINE FUNCTIONS -- #
def load_image(path,targetSize=(224,224),preprocess=True):
    data = kerasImage.load_img(path,target_size=targetSize)
    if preprocess:
        data = kerasImage.img_to_array(data)
        data = np.expand_dims(data,axis=0)
        data = preprocess_input(data)
    return data

def normalize(x):
    return (x+K.epsilon())/(K.sqrt(K.mean(K.square(x)))+K.epsilon())

def deprocess_image(x):
    x = x.copy()
    x -= x.mean(); x /= (x.std() + K.epsilon()); x *= 0.1
    # CLIP DATA
    x += 0.5; x = np.clip(x, 0, 1)
    # CONVERT TO RGB
    x *= 255; x = np.clip(x, 0, 255).astype('uint8')
    return x

In [12]:
# -- DEFINE DECONV LAYER CLASSES -- #
# -- CONV2D DECONOLUTION -- #
class DConv2D(object):
    # -- INITIALIZATION -- #
    def __init__(self,layer):
        self.layer = layer
        weights = layer.get_weights()
        W = weights[0]; b = weights[1]
        # -- UP FUNCTION -- #
        upFilter = W.shape[0]
        upRow = W.shape[2]
        upCol = W.shape[3]
        layerInput = Input(shape = layer.input_shape[1:])
        layerOutput = Conv2D(nb_filter = upFilter,
                             nb_row = upRow,
                             nb_col = upCol, 
                             border_mode = 'same',
                             weights = [W,b])(layerInput)
        self.up_function = K.function([layerInput],[layerOutput])
        # -- DOWN FUNCTION -- #
        W = np.transpose(W,(1,0,2,3))
        W = W[:,:,::-1,::-1]
        downFilter = W.shape[1]
        downRow = W.shape[2]
        downCol = W.shape[3]
        b = np.zeros(downFilter)
        inputLayer = Input(shape = layer.output_shape[1:])
        outputLayer = Conv2D(nb_filter = downFilter,
                             nb_row = downRow,
                             nb_col = downCol,
                             border_mode = 'same',
                             weights = [W,b])(inputLayer)
        self.down_function = K.function([inputLayer],[outputLayer])
    # -- UP PASS -- #
    def up(self,data):
        self.upData = self.up_function([data])
        return self.upData
    # -- DOWN PASS -- #
    def down(self,data):
        self.downData = self.down_function([data])
        return self.downData
    
# -- DENSE DECONVOLUTION -- #
class DDense(object):
    def __init__(self,layer):
        self.layer = layer
        weights = layer.get_weights()
        W = weights[0]
        b = weights[1]
        # -- UP FUNCTION -- #
        inputLayer = Input(shape = layer.input_shape[1:])
        outputLayer = Dense(output_dim = layer.output_shape[1],weights=[W,b])(inputLayer)
        self.up_function = K.function([inputLayer],[outputLayer])
        # -- DOWN FUNCTION -- #
        W = W.transpose()
        b = np.zeros(layer.input_shape[1])
        inputLayer = Input(shape = layer.output_shape[1:])
        outputLayer = Dense(output_dim = layer.input_shape[1],
                            weights = [W,b])(inputLayer)
        self.down_function = K.function([inputLayer],[outputLayer])
    # -- UP PASS -- #
    def up(self,data):
        self.upData = self.up_function([data])
        return self.upData
    # -- DOWN PASS -- #
    def down(self,data):
        self.downData = self.down_function([data])
        return self.downData
    
# -- POOLING LAYERS DECONVOLUTION -- #
class DPooling(object):
    def __init__(self,layer):
        self.layer = layer
        self.poolSize = layer.pool_size
    # -- UP PASS -- #
    def up(self,data):
        [self.upData,self.switch] = self._max_pooling_with_switch(data,self.poolSize)
        return self.upData
    # -- DOWN PASS -- #
    def down(self,data):
        self.downData = self._max_unpooling_with_switch(data,self.switch)
        return self.downData
    # -- FORWARD POOLING -- #
    def _max_pooling_with_switch(self,data,poolSize):
        switch = np.zeros(data.shape)
        outShape = list(data.shape)
        outShape[2] = outShape[2]/poolSize[0]
        outShape[3] = outShape[3]/poolSize[1]
        rowPool = int(poolSize[0])
        colPool = int(poolSize[1])
        pooled = np.zeros(outShape)
        for sample in range(data.shape[0]):
            for dim in range(data.shape[1]):
                for row in range(outShape[2]):
                    for col in range(outShape[3]):
                        patch = data[sample,
                                     dim,
                                     row*rowPool : (row+1)*rowPool,
                                     col*colPool : (col+1)*colPool]
                        maxVal = np.max(patch)
                        pooled[sample,dim,row,col] = maxVal
                        maxRow = np.argmax(np.max(patch,axis=1))
                        maxCol = np.argmax(patch,axis=1)[maxRow]
                        switch[sample,
                               dim,
                               row*rowPool+maxRow,
                               col*colPool+maxCol] = 1
        return [pooled,switch]
    # -- BACKWARD POOLING -- #
    def _max_unpooling_with_switch(self,data,switch):
        tile = np.ones((switch.shape[2]/data.shape[2], switch.shape[3]/data.shape[3]))
        return np.kron(data,tile)*switch

# -- ACTIVATIONS DECONVOLUTIONS -- #
class DActivation(object):
    def __init__(self,layer,linear=False):
        self.layer = layer
        self.linear = linear
        self.activation = layer.activation
        inputLayer = K.placeholder(shape = layer.output_shape)
        outputLayer = self.activation(inputLayer)
        # -- RELU ACTIVATION -- #
        self.up_function = K.function([inputLayer],[outputLayer])
        self.down_function = K.function([inputLayer],[outputLayer])
    def up(self,data):
        self.upData = self.up_function([data])
        return self.upData
    def down(self,data):
        self.downData = self.down_function([data])
        return self.downData
    
# -- FLATTEN DECONVOLUTION -- #
class DFlatten(object):
    def __init__(self,layer):
        self.layer = layer
        self.shape = layer.input_shape[1:]
        self.up_function = K.function([layer.input],[layer.output])
    def up(self,data):
        self.upData = self.up_function([data])
        return self.upData
    def down(self,data):
        shape = [data.shape[0]]+list(self.shape)
        assert np.prod(self.shape) == np.prod(data.shape[1:])
        self.downData = np.reshape(data,shape)
        return self.downData
    
# -- INPUT DECONVOLUTION -- #
class DInput(object):
    def __init__(self,layer):
        self.layer = layer
    def up(self,data):
        self.upData = data
        return self.upData
    def down(self,data):
        self.downData = data
        return self.downData
    
# -- BATCH NORMALIZATION DECONVOLUTION -- #
class DBatchNorm(object):
    def __init__(self,layer):
        self.layer = layer
    def up(self,data):
        self.mean = data.mean()
        self.std = data.std()
        data -= self.mean
        data /= self.std+K.epsilon()
        self.upData = data
        return self.upData
    def down(self,data):
        data += self.mean
        data *= self.std
        self.downData = data
        return self.downData

In [10]:
# -- DEFINE DECONVOLUTION MODEL -- #
def define_deconvolution_model(model,layerName):
    deconvLayers = []
    for layer in model.layers:
        if layer.__class__.__name__ == 'Conv2D':
            deconvLayers.append(DConv2D(layer))
            deconvLayers.append(DActivation(layer))
        elif layer.__class__.__name__ == 'MaxPooling2D':
            deconvLayers.append(DPooling(layer))
        elif layer.__class__.__name__ == 'Dense':
            deconvLayers.append(DDense(layer))
            deconvLayers.append(DActivation(layer))
        elif layer.__class__.__name__ == 'Activation':
            deconvLayers.append(DActivation(layer))
        elif layer.__class__.__name__ == 'Flatten':
            deconvLayers.append(DFlatten(layer))
        elif layer.__class__.__name__ == 'InputLayer':
            deconvLayers.append(DInput(layer))
        elif layer.__class__.__name__ == 'BatchNormalization':
            deconvLayers.append(DBatchNorm(layer))
        else:
            print('Can not handle this type of layer')
            print(layer.get_config())
            sys.exit()
        if layer.name == layerName:
            break
    return deconvLayers

# -- DECONVOLUTION UP & DOWN PASSES -- #
def visualize(data,deconvLayers,featVisualize,visualizeMode='all'):
    # -- FORWARD PASS -- #
    deconvLayers[0].up(data)
    for k in range(1, len(deconvLayers)):
        deconvLayers[k].up(deconvLayers[k-1].upData)
    output = deconvLayers[-1].upData
    assert output.ndim == 2 or output.ndim == 4
    if output.ndim == 2:
        featureMap = output[:,featVisualize]
    else:
        featureMap = output[:, :, :, featVisualize]
    if visualizeMode == 0:
        maxAct = featureMap.max()
        temp = featureMap == maxAct
        featureMap = featureMap*temp
    elif visualizeMode != 1:
        print('Illegal visualize mode')
        sys.exit()
    output = np.zeros_like(output)
    if output.ndim == 2:
        output[:, featVisualize] = featureMap
    else:
        output[:, :, :, featVisualize] = featureMap
    # -- BACKWARD PASS -- #
    deconvLayers[-1].down(output)
    for k in range(len(deconvLayers)-2,-1,-1):
        deconvLayers[k].down(deconvLayers[k+1].downData)
    deconv = deconvLayers[0].downData
    deconv = deconv.squeeze()
    return deconv

In [14]:
# -- MAIN -- #
if not os.path.isdir('./Results/Deconv'):
    os.mkdir('./Results/Deconv')
model = VGG16(weights='imagenet',include_top=True)

In [15]:
# -- USER SELECTION -- #
visualizeMode = int(input('Select the type of Visualize Mode: (0)Max, (1)All:'))
for layer in model.layers:
    print(layer.name)
layerName = input('Select the desired layer:')
deconvLayers = define_deconvolution_model(model,layerName)
numFilt = model.get_layer(layerName).shape[3]
featVisualize =  int(input('Select the filter to visualize: from(0-{0})'.format(numFilt)))
fileName = './Data/images/'+imgSel+'.jpg'
data = load_image(fileName)
deconv = visualize(data,deconvLayers,featVisualize,visualizeMode)
output = deprocess_image(deconv)

Select the type of Visualize Mode: (0)Max, (1)All: 0


input_5
block1_conv1
block1_conv2
block1_pool
block2_conv1
block2_conv2
block2_pool
block3_conv1
block3_conv2
block3_conv3
block3_pool
block4_conv1
block4_conv2
block4_conv3
block4_pool
block5_conv1
block5_conv2
block5_conv3
block5_pool
flatten
fc1
fc2
predictions


Select the desired layer: block5_conv3


/Users/paudomingogregorio/Desktop/Projects/TFG/Code/.env/lib/python3.7/site-packages/ipykernel_launcher.py:18: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(weights=[array([[[..., kernel_size=(3, 64), filters=3, padding="same")`


ValueError: Layer weight shape (3, 64, 3, 3) not compatible with provided weight shape (3, 3, 3, 64)

In [ ]:
# -- PLOTTING RESULTS -- #
plt.imshow(output)
plt.show()
img = kerasImage.array_to_img(output,scale=False)
img.save('./Results/Deconv/'+imgSel+'.png')